<a href="https://colab.research.google.com/github/Flaiba/Coevolution_of_cooperative_lifestyles_and_reduced_cancer_prevalence_in_mammals/blob/main/Model5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Python code for simulating Population Dynamics

This is the code for simulations and plotting of the Mathematical modeling presented in the paper "Coevolution of cooperative lifestyles and low cancer incidence in mammals" authored by Catalina Sierra, Julian Maxwell, Nicolás Flaibani, Constanza Sánchez de la Vega, Alejandra C. Ventura, Nicolás José Lavagnino and Matías Blaustein.

# Define functions for modeling populations, simulating models and plotting results

In [ ]:
import numpy as np
import pandas as pd
import scipy
from scipy.integrate import odeint, ode
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator, MultipleLocator

## Functions

In [ ]:
def frame(param_tuple):
  """
Creates a parameters DataFrame from a tuple.
Must respect given order (see code).
  """
  param_keys = ('$π$', '$ρ$',
               '$κ_{JA1}$', '$κ_{JS1}$', '$κ_{A1}$', '$κ_{S1}$',
               '$α_1$', '$ω_1$', '$β_{A1}$', '$β_{S1}$', '$γ_{A1}$', '$γ_{S1}$', '$σ_1$',
               '$μ_{JA1}$', '$μ_{JS1}$', '$μ_{A1}$', '$μ_{S1}$',
               '$κ_{JA2}$', '$κ_{JS2}$', '$κ_{A2}$', '$κ_{S2}$',
               '$α_2$', '$ω_2$', '$β_{A2}$', '$β_{S2}$', '$γ_{A2}$', '$γ_{S2}$', '$σ_2$',
               '$μ_{JA2}$', '$μ_{JS2}$', '$μ_{A2}$', '$μ_{S2}$')
  dict_param = dict(map(lambda i,j : (i,j) , param_keys, param_tuple))
  param_df = pd.Series(dict_param, dict_param.keys())

  return param_df


In [ ]:
def sol_odeint_df(system, init_cond: list, parameters: tuple, t_fin: float, dt:float, t_ini: float = 0) -> pd.DataFrame:
  """
  Returns 3 DataFrames:
  Simulated Dynamics from time integration by scipy's odeint
  Input parameters
  Input initial values
  """
  t=np.arange(t_ini, t_fin, dt)
  sol = odeint(system, init_cond, t, args = (parameters))


# creates a DataFrame for time course data
  labels = ['R(t)', 'J_A1(t)', 'J_S1(t)', 'A1(t)', 'S1(t)', 'J_A2(t)', 'J_S2(t)', 'A2(t)', 'S2(t)']
  sol_df = pd.DataFrame(data=sol, index=t, columns = labels)

#creates a DataFrame for input parameter values used for simulation
  param_df = frame(parameters)

#creates a DataFrame for input Initial Conditions
  InCond_df = sol_df.iloc[0, :]

  return sol_df, param_df, InCond_df

In [ ]:
def expand_df(sol01: pd.DataFrame, full=False):
  """
  Adds further columns to input DataFrame.
  """
  # size of subpopulation 1
  sol01['N1(t)'] = sol01.iloc[:,[1,2,3,4]].sum(1)
  # size of subpopulation 2
  sol01['N2(t)'] = sol01.iloc[:,[5,6,7,8]].sum(1)
  # total consumer population size
  sol01['N(t)'] = sol01.iloc[:,[9,10]].sum(1)
  #fraction of each subpopulation over total consumer population
  sol01[['n1(t)', 'n2(t)']] = sol01[['N1(t)','N2(t)']].div(sol01['N(t)'], axis=0)

  if full == True:

    #sum of adult-born juveniles from both subpopulations
    sol01['J_A(t)']= sol01[['J_A1(t)', 'J_A2(t)']].sum(1)
    #sum of senior-born juveniles from both subpopulations
    sol01['J_S(t)']= sol01[['J_S1(t)', 'J_S2(t)']].sum(1)
    #sum of kinds of juveniles from the same subpopulation (1)
    sol01['J1(t)'] = sol01.iloc[:,[1,2]].sum(1)
    #sum of kinds of juveniles from the same subpopulation (2)
    sol01['J2(t)'] = sol01.iloc[:,[5,6]].sum(1)
    #all juveniles (regardless of origin) summed
    sol01['J(t)']  = sol01[['J_A(t)', 'J_S(t)']].sum(1)
    # sum of all adults in metapopulation
    sol01['A(t)']  = sol01[['A1(t)', 'A2(t)']].sum(1)
    # sum of all seniors in metapopulation
    sol01['S(t)']  = sol01[['S1(t)', 'S2(t)']].sum(1)

    #fraction of adults to total size in a single subpopulation (1)
    sol01['a1(t)'] = sol01['A1(t)'].div(sol01['N1(t)'], axis=0)
    #fraction of adults to total size in a single subpopulation (2)
    sol01['a2(t)'] = sol01['A2(t)'].div(sol01['N2(t)'], axis=0)
    #fraction of all adults in total metapopulation
    sol01['a(t)']  = sol01['A(t)'].div(sol01['N(t)'], axis=0)


  return sol01

In [ ]:
def mean_df(sol01: pd.DataFrame):
  """
  Calculates the mean value for last fifth of simulated time.
  Useful to compare to the last value of simulation.
  """
  df1 = pd.DataFrame(sol01.iloc[-len(sol01)//5:,:].mean(0)).T
  df1.index=['mean']

  return df1

In [ ]:
def function(system, start, end, step, param = tuple([0]*29), init_cond = [0]*9,
             t_fin = 0, dt = 0, control1='$μ_{S1}$', control2='$μ_{S1}$', full = True):
  """
  This function returns 2 lists (one of floats, another one of dictionaries)
  from the following input parameters

  System: must be a defined model (MetaPopX)

  (np.arange parameters)
  start: Lower bound for control parameter range
  end: Upper bound for control parameter range (not included)
  step: step of the control parameter range

  param: of paramater set values for system
  init_cond: list of initial values for system state variables

  t_fin: end time of simulation (total time)
  dt: time step for time course returned

  control1: keyword for selected control parameter
  control2: keyword for additonal control parameter

  full: False to get less information

  Floats in fifrst list are the control variables values
  in the order used for simulations.
  Dictonaries in second list provide all the information
  from the multiple simulations ran by this function.
  """
  mu_s_array=[]
  dict_sol_array = []

  for i in np.arange(start, end, step):
  # for i in np.linspace(start, end, step): #alternative
    i = round(i, 2)
    param[control1]=i
    param[control2]=i
    #uses sol_odeint_df to create 3 dataframes:
    #time course, input parameters and input initial values
    sol, param, Init_Cond = sol_odeint_df(system, list(init_cond[0:9]),
                                          tuple(param), t_fin=t_fin, dt=dt)
    expand_df(sol, full) #expands the dataframe for additional information
    #adds to the ouput list all data of current simulation in dictioanary form
    dict_sol_array.append({'control_name1': control1, 'control_name2': control2,
                           'parameter_value1': i, 'parameter_value2': i,
                           'last_value': sol.iloc[-1, :], 'mean_value': mean_df(sol),
                           'dynamics': sol, 'params': param, 'Init_Cond': Init_Cond})
    mu_s_array.append(i)

    # i+=step

  return mu_s_array, dict_sol_array


## Systems of Equations

The following systems of ODE are 4 versions of the same model. Variation can be found in the location of the cooperative/competitive term, herein refered to as the $ h $ function.

$$
h = \frac{(1 + \alpha_1 S_1 + \alpha_2 S_2)(1 + \rho R^2)}{{1 + (\omega_1 S_1 + \omega_2 S_2)}}
$$

These models are used for simulating populations in *Modeling cancer and population size* subsection of the paper's Results section and also in the *Extended Data Figures* of the Supplementary Information.

All systems of equations have an unstructered *Resource* dynamics equation and 2 "copies" of the structured *Consumer* population dynamics. This allows us to show the time evolution of a meta-population composed of two subpopulations (with two different sets of life history parameter values) competing for a common pool of resources (*Resource* population). When considering a single population, same initial conditions and parameter values are selected for both "subpopulations".

Additionally, in these models there 2 kinds of Juveniles in each *Consumer* population, those that are born from *Adults* (`J_A1` and `J_A2`), and those that are born from *Senior* adults (`J_S1` and `J_S2`).



In [ ]:
def MetaPopA(V: list,
             t,
             p, r,
             kja1, kjs1, ka1, ks1, a1, w1, ba1, bs1, ga1, gs1, s1, mja1, mjs1, ma1, ms1,
             kja2, kjs2, ka2, ks2, a2, w2, ba2, bs2, ga2, gs2, s2, mja2, mjs2, ma2, ms2) -> list:
    """
Model based on Extended Data Eqs. 4a-d. Here cooperativity/competition function is affecting Juvenile mortality.
This funtion should use the same set of parameters for both subpopulations (1 and 2) if it is simulating a single population.
This function receives an initial state for the system variables V in list form.
It returns the rate of change of the system variables in list form.

Except for V = [Ri, J_A1i, ..., S2i], every other input should be of type float.
    """
    R = V[0]
    J_A1 = V[1]
    J_S1 = V[2]
    A1 = V[3]
    S1 = V[4]
    J_A2 = V[5]
    J_S2 = V[6]
    A2 = V[7]
    S2 = V[8]

    h = (1 + a1*S1 + a2*S2)/(1 + (w1*S1 + w2*S2)/(1 + r*(R**2)))

    dRdt = p - kja1*R*J_A1 + kjs1*R*J_S1 - ka1*R*A1 - ks1*R*S1 - kja2*R*J_A2 + kjs2*R*J_S2 - ka2*R*A2 - ks2*R*S2

    dJ_A1dt = (ba1*(ka1*R))*A1 - (ga1*(kja1*R))*J_A1 - (mja1)*J_A1/h
    dJ_S1dt = (bs1*(ks1*R))*S1 - (gs1*(kjs1*R))*J_S1 - (mjs1)*J_S1/h
    dA1dt = (ga1*(kja1*R))*J_A1 + (gs1*(kjs1*R))*J_S1 - s1*A1 - ma1*A1
    dS1dt = s1*A1 - ms1*S1

    dJ_A2dt = (ba2*(ka2*R))*A2 - (ga2*(kja2*R))*J_A2 - (mja2)*J_A2/h
    dJ_S2dt = (bs2*(ks2*R))*S2 - (gs2*(kjs2*R))*J_S2 - (mjs2)*J_S2/h
    dA2dt = (ga2*(kja2*R))*J_A2 + (gs2*(kjs2*R))*J_S2 - s2*A2 - ma2*A2
    dS2dt = s2*A2 - ms2*S2

    return [dRdt, dJ_A1dt, dJ_S1dt, dA1dt, dS1dt, dJ_A2dt, dJ_S2dt, dA2dt, dS2dt]


In [ ]:
def MetaPopB(V: list,
             t,
             p, r,
             kja1, kjs1, ka1, ks1, a1, w1, ba1, bs1, ga1, gs1, s1, mja1, mjs1, ma1, ms1,
             kja2, kjs2, ka2, ks2, a2, w2, ba2, bs2, ga2, gs2, s2, mja2, mjs2, ma2, ms2):
    """
Model based on Extended Data Eqs. E1a-d. Here cooperativity/competition function is affecting Juvenile foraging.
This funtion should use the same set of parameters for both subpopulations (1 and 2) if it is simulating a single population.
This function receives an initial state for the system variables V in list form.
It returns the rate of change of the system variables in list form.

Except for V = [Ri, J_A1i, ..., S2i], every other input should be of type float.
    """
    R = V[0]
    J_A1 = V[1]
    J_S1 = V[2]
    A1 = V[3]
    S1 = V[4]
    J_A2 = V[5]
    J_S2 = V[6]
    A2 = V[7]
    S2 = V[8]

    h = (1 + a1*S1 + a2*S2)/(1 + (w1*S1 + w2*S2)/(1 + r*(R**2)))

    dRdt = p - ((kja1*h)*R*J_A1)+ ((kjs1*h)*R*J_S1) - ka1*R*A1 - ks1*R*S1 - ((kja2*h)*R*J_A2) + ((kjs2*h)*R*J_S2) - ka2*R*A2 - ks2*R*S2

    dJ_A1dt = (ba1*(ka1*R))*A1 - (ga1*((kja1*h)*R))*J_A1 - (mja1)*J_A1
    dJ_S1dt = (bs1*(ks1*R))*S1 - (gs1*((kjs1*h)*R))*J_S1 - (mjs1)*J_S1
    dA1dt = (ga1*((kja1*h)*R))*J_A1 + (gs1*((kjs1*h)*R))*J_S1 - s1*A1 - ma1*A1
    dS1dt = s1*A1 - ms1*S1

    dJ_A2dt = (ba2*(ka2*R))*A2 - (ga2*((kja2*h)*R))*J_A2 - (mja2)*J_A2
    dJ_S2dt = (bs2*(ks2*R))*S2 - (gs2*((kjs2*h)*R))*J_S2 - (mjs2)*J_S2
    dA2dt = (ga2*((kja2*h)*R))*J_A2 + (gs2*((kjs2*h)*R))*J_S2 - s2*A2 - ma2*A2
    dS2dt = s2*A2 - ms2*S2

    return [dRdt, dJ_A1dt, dJ_S1dt, dA1dt, dS1dt, dJ_A2dt, dJ_S2dt, dA2dt, dS2dt]


In [ ]:
def MetaPopC(V: list,
             t,
             p, r,
             kja1, kjs1, ka1, ks1, a1, w1, ba1, bs1, ga1, gs1, s1, mja1, mjs1, ma1, ms1,
             kja2, kjs2, ka2, ks2, a2, w2, ba2, bs2, ga2, gs2, s2, mja2, mjs2, ma2, ms2):
    """
Model based on Extended Data Eqs. E2a-d. Here cooperativity/competition function is affecting Juvenile to Adult maturation.
This funtion should use the same set of parameters for both subpopulations (1 and 2) if it is simulating a single population.
This function receives an initial state for the system variables V in list form.
It returns the rate of change of the system variables in list form.

Except for V = [Ri, J_A1i, ..., S2i], every other input should be of type float.
    """
    R = V[0]
    J_A1 = V[1]
    J_S1 = V[2]
    A1 = V[3]
    S1 = V[4]
    J_A2 = V[5]
    J_S2 = V[6]
    A2 = V[7]
    S2 = V[8]

    h = (1 + a1*S1 + a2*S2)/(1 + (w1*S1 + w2*S2)/(1 + r*(R**2)))

    dRdt = p - kja1*R*J_A1 + kjs1*R*J_S1 - ka1*R*A1 - ks1*R*S1 - kja2*R*J_A2 + kjs2*R*J_S2 - ka2*R*A2 - ks2*R*S2

    dJ_A1dt = (ba1*(ka1*R))*A1 - ((ga1*h)*(kja1*R))*J_A1 - (mja1)*J_A1
    dJ_S1dt = (bs1*(ks1*R))*S1 - ((gs1*h)*(kjs1*R))*J_S1 - (mjs1)*J_S1
    dA1dt = ((ga1*h)*(kja1*R))*J_A1 + ((gs1*h)*(kjs1*R))*J_S1 - s1*A1 - ma1*A1
    dS1dt = s1*A1 - ms1*S1

    dJ_A2dt = (ba2*(ka2*R))*A2 - ((ga2*h)*(kja2*R))*J_A2 - (mja2)*J_A2
    dJ_S2dt = (bs2*(ks2*R))*S2 - ((gs2*h)*(kjs2*R))*J_S2 - (mjs2)*J_S2
    dA2dt = ((ga2*h)*(kja2*R))*J_A2 + ((gs2*h)*(kjs2*R))*J_S2 - s2*A2 - ma2*A2
    dS2dt = s2*A2 - ms2*S2

    return [dRdt, dJ_A1dt, dJ_S1dt, dA1dt, dS1dt, dJ_A2dt, dJ_S2dt, dA2dt, dS2dt]


In [ ]:
def MetaPopD(V: list,
             t,
             p, r,
             kja1, kjs1, ka1, ks1, a1, w1, ba1, bs1, ga1, gs1, s1, mja1, mjs1, ma1, ms1,
             kja2, kjs2, ka2, ks2, a2, w2, ba2, bs2, ga2, gs2, s2, mja2, mjs2, ma2, ms2):
    """
Model based on Extended Data Eqs. E3a-d. Here cooperativity/competition function is affecting Adult and Senior fertility.
This funtion should use the same set of parameters for both subpopulations (1 and 2) if it is simulating a single population.
This function receives an initial state for the system variables V in list form.
It returns the rate of change of the system variables in list form.

Except for V = [Ri, J_A1i, ..., S2i], every other input should be of type float.
    """
    R = V[0]
    J_A1 = V[1]
    J_S1 = V[2]
    A1 = V[3]
    S1 = V[4]
    J_A2 = V[5]
    J_S2 = V[6]
    A2 = V[7]
    S2 = V[8]

    h = (1 + a1*S1 + a2*S2)/(1 + (w1*S1 + w2*S2)/(1 + r*(R**2)))

    dRdt = p - kja1*R*J_A1 + kjs1*R*J_S1 - ka1*R*A1 - ks1*R*S1 - kja2*R*J_A2 + kjs2*R*J_S2 - ka2*R*A2 - ks2*R*S2

    dJ_A1dt = ((ba1*h)*(ka1*R))*A1 - (ga1*(kja1*R))*J_A1 - (mja1)*J_A1
    dJ_S1dt = ((bs1*h)*(ks1*R))*S1 - (gs1*(kjs1*R))*J_S1 - (mjs1)*J_S1
    dA1dt = (ga1*(kja1*R))*J_A1 + (gs1*(kjs1*R))*J_S1 - s1*A1 - ma1*A1
    dS1dt = s1*A1 - ms1*S1

    dJ_A2dt = ((ba2*h)*(ka2*R))*A2 - (ga2*(kja2*R))*J_A2 - (mja2)*J_A2
    dJ_S2dt = ((bs2*h)*(ks2*R))*S2 - (gs2*(kjs2*R))*J_S2 - (mjs2)*J_S2
    dA2dt = (ga2*(kja2*R))*J_A2 + (gs2*(kjs2*R))*J_S2 - s2*A2 - ma2*A2
    dS2dt = s2*A2 - ms2*S2

    return [dRdt, dJ_A1dt, dJ_S1dt, dA1dt, dS1dt, dJ_A2dt, dJ_S2dt, dA2dt, dS2dt]
